In [1]:
%load_ext autoreload
%autoreload 2

import os
from os.path import basename, dirname
from shutil import copyfile

import json
import pickle
from collections import OrderedDict

import torch

os.chdir("..")

from da.fsmt.modeling_fsmt import FSMTForConditionalGeneration
from da.fsmt.tokenization_fsmt import FSMTTokenizer

from types import MethodType

from da.greedy_search_interpret import greedy_search_interpret

In [2]:
main_name = "concat60"
hf_dir = f"experiments/en_et_{main_name}/hf"

tokenizer_hf = FSMTTokenizer.from_pretrained(hf_dir)
model_hf = FSMTForConditionalGeneration.from_pretrained(hf_dir)
model_hf = model_hf.cuda()
model_hf.greedy_search = MethodType(greedy_search_interpret, model_hf)

In [3]:
data = ["▁how ▁would ▁you ▁like ▁that ▁done ▁to ▁you ?", 
"▁- ▁if ▁you ▁have ▁ever ▁had ▁chemotherapy ▁with ▁a ▁drug ▁called ▁doxorubicin ▁or ▁a ▁drug ▁related ▁to", 
"▁it ▁is ▁one ▁of ▁the ▁most ▁difficult ▁jobs ▁in ▁the ▁European ▁Union ▁that ▁he ▁has ▁to ▁do ."] 


In [4]:
data_tok = tokenizer_hf.batch_encode_plus(
        data,
        padding="longest", 
        return_tensors="pt",
        return_token_type_ids=False,
        return_attention_mask=True
    )

In [5]:
for k, v in data_tok.items():
    data_tok[k] = v.to(model_hf.device)

In [7]:
res_generate = model_hf.generate(**data_tok,
                   #return_dict=True,
                   output_hidden_states=True,
                   output_attentions=True,
                   do_sample=False,
                   num_beams=1)

In [8]:
res_forward = model_hf.forward(**data_tok,
                   return_dict=True,
                   output_hidden_states=True,
                   output_attentions=True)

In [29]:
encoder = model_hf.base_model.encoder

res_encoder = encoder.forward(**data_tok,
                   return_dict=True,
                   output_hidden_states=True,
                   output_attentions=True)

In [21]:
res_encoder.keys()

odict_keys(['last_hidden_state', 'hidden_states', 'attentions'])

In [9]:
for a,b in zip(res_generate['encoder_hidden_states'], res_forward['encoder_hidden_states']):
    assert torch.all(torch.eq(a,b)).item()
    print(torch.all(torch.eq(a,b)).item())

True
True
True
True
True
True
True


In [35]:
for a,b in zip(res_generate['encoder_hidden_states'], res_encoder['hidden_states']):
    assert torch.all(torch.eq(a,b)).item()
    print(torch.all(torch.eq(a,b)).item())

True
True
True
True
True
True
True


In [ ]:
res